In [13]:
# -----------------------------------------------------------------------------
# RESOURCE USAGE ANALYSIS FOR CLIMATE MODEL JOBS
# -----------------------------------------------------------------------------

import pandas as pd
import numpy as np
# --- New Import for Plotting ---
import plotly.express as px
# 1. Load the Data
# Assumes the file 'process_variable_frequency_usage_data (1).csv' is in the
# same directory as this notebook.
file_path = '/mnt/share/homes/bcreiner/repos/idd-climate-models/notebooks/process_variable_frequency_usage_data (1).csv'
df = pd.read_csv(file_path)

# Ensure numeric columns are treated as such
df['requested_memory_gib'] = df['requested_memory_gib'].astype(float)
df['memory_gib'] = df['memory_gib'].astype(float)
df['requested_runtime_seconds'] = df['requested_runtime_seconds'].astype(float)
df['runtime_seconds'] = df['runtime_seconds'].astype(float)

In [15]:
# ==============================================================================
# 2. Recreate Resource Estimation Logic (Reverse Engineering)
# ==============================================================================

# Constants from the original get_resource_info function
REQUIRED_MEM_FACTOR = 4.0
MIN_MEM_GB = 8.0
MAX_MEM_GB = 80.0
FIXED_OVERHEAD_GB = 4.0
MAX_REVERSE_ENGINEERED_SIZE_GB = (MAX_MEM_GB - FIXED_OVERHEAD_GB) / REQUIRED_MEM_FACTOR # 19.0 GB

def estimate_file_size_gb(requested_mem_gib):
    """
    Estimates the original size_gb of the input file(s) by reversing the
    resource calculation. This is used only for general binning and plotting.
    """
    estimated_size = (requested_mem_gib - FIXED_OVERHEAD_GB) / REQUIRED_MEM_FACTOR
    
    # Handle the capped low end: if requested was 8GB, the size_gb was <= 1.0
    low_cap_mask = requested_mem_gib <= MIN_MEM_GB
    estimated_size[low_cap_mask] = 1.0
    
    # Handle the capped high end: use the reverse value if it is <= the max accurate size
    # All requests >= MAX_MEM_GB are grouped by the single max size bin in plotting.
    
    return estimated_size

df['estimated_size_gb'] = estimate_file_size_gb(df['requested_memory_gib'])


# ==============================================================================
# 3. Resource Assessment and Categorization (Refined Logic)
# ==============================================================================

# Define Thresholds
MEM_BUFFER = 0.01  # 10MB buffer for OOM trigger
TIME_BUFFER_RATIO = 0.98 # If runtime_seconds is > 98% of requested, it's a likely time-out.

def categorize_failure(row):
    is_failed = row['status'] == 'Z'

    if not is_failed:
        return "Successful" 
    
    # --- ASSUMPTION: ALL 'Z' FAILURES MUST BE ASSIGNED TO MEMORY OR TIME ---
    
    # 1. Calculate Ratios
    # Memory Ratio: How much of the requested memory was utilized (or exceeded)
    mem_ratio = row['memory_gib'] / row['requested_memory_gib']
    # Time Ratio: How much of the requested time was utilized
    time_ratio = row['runtime_seconds'] / row['requested_runtime_seconds']
    
    # 2. Check for Trigger
    is_oom_trigger = mem_ratio > 1.0 + MEM_BUFFER
    is_timeout_trigger = time_ratio > TIME_BUFFER_RATIO
    
    # 3. Determine 'Closeness to Limit' for tie-breaker/assignment
    
    # Normalization: Time ratio is normalized so that TIME_BUFFER_RATIO is the limit.
    # The 'closeness' is the ratio of actual utilization to the failure threshold.
    time_closeness_to_limit = time_ratio / TIME_BUFFER_RATIO 

    # If the job is an ambiguous 'Z' (didn't clearly hit either threshold), 
    # we assign it to the resource that was utilized relatively more.
    if not is_oom_trigger and not is_timeout_trigger:
        if mem_ratio >= time_closeness_to_limit:
            return "Failed (Memory)"
        else:
            return "Failed (Timeout)"
            
    # 4. Apply the tie-breaker for clear hits
    if is_oom_trigger and is_timeout_trigger:
        # Compare which resource exceeded its failure threshold relatively more.
        mem_excess = mem_ratio - 1.0
        time_excess = time_ratio - TIME_BUFFER_RATIO
        
        if mem_excess >= time_excess:
            return "Failed (Memory)"
        else:
            return "Failed (Timeout)"
            
    # 5. If only one is triggered, that's the cause
    elif is_oom_trigger:
        return "Failed (Memory)"
    elif is_timeout_trigger:
        return "Failed (Timeout)"

    # Should be unreachable given the logic above, but included for safety.
    return "Failed (Other/Unknown - Error in Logic)" 


df['failure_cause'] = df.apply(categorize_failure, axis=1)

# ==============================================================================
# 4. Table 2: Actionable Failure Analysis (Custom Bins)
# ==============================================================================

# Define the data to be analyzed for Table 2
analysis_df = df.copy()

def get_memory_regime(requested_mem):
    """Custom binning focusing on exact Min/Max and the Mid-Range."""
    if requested_mem == MIN_MEM_GB:
        return "A. Exactly 8 GB (Min Cap)"
    elif requested_mem == MAX_MEM_GB:
        return "E. Exactly 80 GB (Max Cap)"
    elif requested_mem < 16.0:
        return "B. 8 < Request < 16 GB"
    elif requested_mem < 44.0:
        return "C. 16 - 44 GB (Mid-Range)"
    elif requested_mem < MAX_MEM_GB:
        return "D. 44 - 80 GB (Upper Range)"
    else:
        # Should not happen if MAX_MEM_GB is 80.0
        return "Z. Invalid Request (>80 GB)"

analysis_df['memory_regime'] = analysis_df['requested_memory_gib'].apply(get_memory_regime)

# Drop the invalid request row if it exists
analysis_df = analysis_df[analysis_df['memory_regime'] != 'Z. Invalid Request (>80 GB)']

# Map the failure_cause to the desired column names for the crosstab
analysis_df['Outcome'] = analysis_df['failure_cause'].apply(lambda x: 'Successful' if x == 'Successful' else ('Failed (Memory)' if 'Memory' in x else ('Failed (Timeout)' if 'Timeout' in x else 'Failed (Other)')))

# Create the counts pivot table
regime_crosstab = pd.crosstab(
    analysis_df['memory_regime'],
    analysis_df['Outcome'],
    dropna=False
).fillna(0).astype(int)

# --- Q1: Adding Success Column and Percentages ---

# 1. Ensure all columns are present (even if zero)
for col in ['Successful', 'Failed (Memory)', 'Failed (Timeout)', 'Failed (Other)']:
    if col not in regime_crosstab.columns:
        regime_crosstab[col] = 0

# 2. Calculate Totals
regime_crosstab['Total Attempts'] = regime_crosstab[['Successful', 'Failed (Memory)', 'Failed (Timeout)', 'Failed (Other)']].sum(axis=1)

# 3. Calculate Percentages
regime_crosstab['% Failed (Memory)'] = (regime_crosstab['Failed (Memory)'] / regime_crosstab['Total Attempts'] * 100).round(1)
regime_crosstab['% Failed (Timeout)'] = (regime_crosstab['Failed (Timeout)'] / regime_crosstab['Total Attempts'] * 100).round(1)

# 4. Clean up and rename index
regime_crosstab = regime_crosstab.sort_index()
regime_crosstab.index = [label[3:] for label in regime_crosstab.index]

# 5. Select final columns for Table 2
table_2_columns = [
    'Total Attempts',
    'Successful',
    'Failed (Memory)',
    'Failed (Timeout)',
    '% Failed (Memory)',
    '% Failed (Timeout)'
]
table_2 = regime_crosstab[table_2_columns]


# ==============================================================================
# 5. Plotting Data Prep (Q2: Stacked Area Plot)
# ==============================================================================

# Use estimated_size_gb for the X-axis for the plot
plot_df = df.copy()

# Define size bins for the plot X-axis (Increased bins, capped at 19 GB)
# Bins: 0, 1, 2, 4, 8, 12, 16, 19, Inf
size_bins = [0, 1, 2, 4, 8, 12, 16, MAX_REVERSE_ENGINEERED_SIZE_GB, np.inf]
size_labels = ['<1 GB', '1-2 GB', '2-4 GB', '4-8 GB', '8-12 GB', '12-16 GB', '16-19 GB (Capped)', '>19 GB (Capped)']

plot_df['size_regime'] = pd.cut(
    plot_df['estimated_size_gb'],
    bins=size_bins,
    labels=size_labels,
    right=True,
    include_lowest=True
)

# Re-categorize outcome for plotting (Success, Mem, Time, Other)
plot_df['Outcome'] = plot_df['failure_cause'].apply(lambda x: x if 'Failed' in x else 'Successful')

# Create the plotting pivot table: count of outcomes per size regime
plot_data = plot_df.groupby(['size_regime', 'Outcome']).size().unstack(fill_value=0)

# Calculate percentages for stacking (essential for "Percent of Jobs" Y-axis)
plot_data_percent = plot_data.div(plot_data.sum(axis=1), axis=0) * 100

# Order the columns for consistent stacking: Success at the bottom
plot_columns_order = ['Successful', 'Failed (Timeout)', 'Failed (Memory)', 'Failed (Other/Unknown)']
for col in plot_columns_order:
    if col not in plot_data_percent.columns:
        plot_data_percent[col] = 0.0

plot_data_percent = plot_data_percent[plot_columns_order]

# --- PLOTTING CODE ADDED HERE ---
# Reshape the data for Plotly Express (melting the pivot table)
plot_data_melted = plot_data_percent.reset_index().melt(
    id_vars='size_regime', 
    value_vars=plot_columns_order,
    var_name='Outcome Type', 
    value_name='Percent of Jobs'
)

# Define custom colors
color_map = {
    'Successful': 'rgb(44, 160, 44)',         # Green
    'Failed (Timeout)': 'rgb(255, 127, 14)',  # Orange
    'Failed (Memory)': 'rgb(214, 39, 40)',    # Red
    'Failed (Other/Unknown)': 'rgb(127, 127, 127)' # Grey
}

# Ensure the categories are ordered for the X-axis
ordered_size_regimes = plot_data_percent.index.tolist()
plot_data_melted['size_regime'] = pd.Categorical(plot_data_melted['size_regime'], categories=ordered_size_regimes, ordered=True)
plot_data_melted = plot_data_melted.sort_values('size_regime')


fig = px.area(
    plot_data_melted, 
    x='size_regime', 
    y='Percent of Jobs', 
    color='Outcome Type',
    title='Job Outcome Distribution by Estimated File Size Regime (Increased Resolution)',
    labels={'size_regime': 'Estimated File Size Regime (GB)', 'Percent of Jobs': 'Percent of Total Attempts'},
    color_discrete_map=color_map,
    category_orders={"Outcome Type": plot_columns_order, "size_regime": ordered_size_regimes}
)

# Customize the plot appearance
fig.update_layout(
    xaxis_title='Estimated File Size Regime (GB)',
    yaxis_title='Percent of Jobs',
    yaxis=dict(tickformat=".0f", range=[0, 100]), # Ensure Y-axis goes from 0 to 100
    legend_title_text='Job Outcome'
)

# Display the plot
fig.show()

# ----------------------------------------
# END PLOTTING CODE
# ----------------------------------------


# ==============================================================================
# 6. Output
# ==============================================================================

print("--- Resource Usage Analysis Summary ---")

# Summary Table by Category
summary_by_category = df['failure_cause'].value_counts().reset_index()
summary_by_category.columns = ['Failure Cause', 'Count']
print("\n[Table 1] Overall Summary of Job Attempts by Primary Cause:")
print(summary_by_category.to_markdown(index=False, numalign="left", stralign="left"))


print("\n[Table 2] Failures Broken Down by REQUESTED Memory and Failure Type (Actionable Bins):")
print(table_2.to_markdown(numalign="left", stralign="left"))

print("\n\n[Plotting Data] Stacked Area Plot Data by Estimated File Size (used for plot above):")
print("This table shows the distribution of job outcomes (Success/Failure) across file size regimes.")
print("X-axis: Index, Y-axis: Percent.")
print(plot_data_percent.to_markdown(numalign="left", stralign="left"))


# Show examples of the key failure modes
print("\n--- Examples of Key Resource Issues ---")

# OOM Examples
print("\n[Example] Failed (Memory):")
mem_failure_example = df[df['failure_cause'].str.contains('Memory')]
if not mem_failure_example.empty:
    print(mem_failure_example.head(2)[['task_id', 'estimated_size_gb', 'requested_memory_gib', 'memory_gib', 'requested_runtime_seconds', 'runtime_seconds']].to_markdown(index=False, numalign="left", stralign="left"))

# Timeout Examples
print("\n[Example] Failed (Timeout):")
timeout_example = df[df['failure_cause'].str.contains('Timeout')]
if not timeout_example.empty:
    print(timeout_example.head(2)[['task_id', 'estimated_size_gb', 'requested_memory_gib', 'memory_gib', 'requested_runtime_seconds', 'runtime_seconds']].to_markdown(index=False, numalign="left", stralign="left"))
    
# Wasted Memory Examples (reusing old logic for general waste visualization)
# This uses the less precise 'wastes_mem' flag from the old logic for illustration
df['wastes_mem'] = (df['memory_gib'] * REQUIRED_MEM_FACTOR < df['requested_memory_gib'])
wasted_mem_example = df[(df['status'] != 'Z') & (df['wastes_mem'])]
if not wasted_mem_example.empty:
    print("\n[Example] Wasted Resources (Completed):")
    print(wasted_mem_example.head(2)[['task_id', 'estimated_size_gb', 'requested_memory_gib', 'memory_gib', 'requested_runtime_seconds', 'runtime_seconds']].to_markdown(index=False, numalign="left", stralign="left"))

/tmp/ipykernel_3507298/3250637053.py:186: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



--- Resource Usage Analysis Summary ---

[Table 1] Overall Summary of Job Attempts by Primary Cause:
| Failure Cause    | Count   |
|:-----------------|:--------|
| Successful       | 2096    |
| Failed (Memory)  | 1126    |
| Failed (Timeout) | 215     |

[Table 2] Failures Broken Down by REQUESTED Memory and Failure Type (Actionable Bins):
|                          | Total Attempts   | Successful   | Failed (Memory)   | Failed (Timeout)   | % Failed (Memory)   | % Failed (Timeout)   |
|:-------------------------|:-----------------|:-------------|:------------------|:-------------------|:--------------------|:---------------------|
| Exactly 8 GB (Min Cap)   | 1548             | 1548         | 0                 | 0                  | 0                   | 0                    |
| 8 < Request < 16 GB      | 229              | 65           | 131               | 33                 | 57.2                | 14.4                 |
| 16 - 44 GB (Mid-Range)   | 1157             | 131         

In [10]:
76/4

19.0

In [2]:
# ==============================================================================
# 2. Recreate Resource Estimation Logic (Reverse Engineering)
# ==============================================================================

# Constants from the original get_resource_info function
REQUIRED_MEM_FACTOR = 4.0
MIN_MEM_GB = 8.0
MAX_MEM_GB = 80.0
FIXED_OVERHEAD_GB = 4.0

def estimate_file_size_gb(requested_mem_gib):
    """
    Estimates the original size_gb of the input file(s) by reversing the
    resource calculation:
    requested_mem_gib = int(size_gb * 4.0) + 4
    => size_gb ~ (requested_mem_gib - 4) / 4.0
    
    This function handles the MIN/MAX caps for interpretability.
    """
    # Create a Series of estimated sizes
    estimated_size = (requested_mem_gib - FIXED_OVERHEAD_GB) / REQUIRED_MEM_FACTOR
    
    # Handle the capped low end: if requested was 8GB, the size_gb was <= 1.0
    # Use 1.0 as a floor for this regime for better categorization.
    low_cap_mask = requested_mem_gib <= MIN_MEM_GB
    estimated_size[low_cap_mask] = 1.0
    
    # Handle the capped high end: if requested was 80GB, the size_gb was >= 19.0
    # Use the calculated value if it's over the cap, to allow distinction.
    high_cap_mask = requested_mem_gib >= MAX_MEM_GB
    # For capped jobs, we keep the size estimate as if the cap didn't exist 
    # to categorize the true file size better.
    estimated_size[high_cap_mask] = (requested_mem_gib[high_cap_mask] - FIXED_OVERHEAD_GB) / REQUIRED_MEM_FACTOR
    
    return estimated_size

df['estimated_size_gb'] = estimate_file_size_gb(df['requested_memory_gib'])

In [3]:
# ==============================================================================
# 3. Resource Assessment and Categorization
# ==============================================================================

# Define Thresholds
# Memory: Used memory > Requested memory (over-provisioning needed)
# Time: Actual runtime > Requested runtime (timeout likely) or Used time << Requested time (too much requested)

# A small buffer is added to account for system reporting variance.
MEM_BUFFER = 0.01  # 10MB buffer
TIME_BUFFER_RATIO = 0.98 # If runtime_seconds is > 98% of requested, it's a likely time-out.
TIME_WASTE_RATIO = 0.50 # If runtime_seconds is < 50% of requested, there's significant time waste.

# --- Memory Assessment ---
# Too Little Memory: Used memory exceeds requested memory (OOM Kill likely)
df['needs_more_mem'] = (df['memory_gib'] > df['requested_memory_gib'] + MEM_BUFFER)
# Too Much Memory: Requested is significantly higher than used (Wasted resources)
df['wastes_mem'] = (df['memory_gib'] * REQUIRED_MEM_FACTOR < df['requested_memory_gib'])

# --- Time Assessment ---
# Too Little Time: Job ran for nearly the full requested time (Timeout likely)
# Note: Time-out is confirmed if status='Z' or similar, but this captures potential time-outs.
df['needs_more_time'] = (df['runtime_seconds'] > df['requested_runtime_seconds'] * TIME_BUFFER_RATIO)
# Too Much Time: Job finished quickly, wasting the remaining requested time
df['wastes_time'] = (df['runtime_seconds'] < df['requested_runtime_seconds'] * TIME_WASTE_RATIO)

# --- Categorization ---
def categorize_resource_issue(row):
    # Failure condition based on user clarification (Z=OOM kill/failure)
    is_failed = row['status'] == 'Z'
    
    # Primary Failure Modes
    is_oom = is_failed and row['needs_more_mem']
    is_timeout = is_failed and row['needs_more_time'] and not is_oom
    
    # Cases where the job successfully completed (status != 'Z')
    if not is_failed:
        if row['wastes_mem'] and row['wastes_time']:
            return "Wastes Both (Completed)"
        elif row['wastes_mem']:
            return "Wastes Memory (Completed)"
        elif row['wastes_time']:
            return "Wastes Time (Completed)"
        else:
            return "Efficient (Completed)"
    
    # Primary Failure Cases (Status 'Z')
    if is_oom and is_timeout:
        return "Failed (OOM & Timeout)" # Hit memory limit while timing out
    elif is_oom:
        return "Failed (OOM Kill)"
    elif is_timeout:
        return "Failed (Timeout)"
    
    # Failed for other reasons (e.g., code error, external issue)
    return "Failed (Other/Unknown)"

df['resource_category'] = df.apply(categorize_resource_issue, axis=1)

In [4]:
# ==============================================================================
# 4. Summary and Visualization Prep
# ==============================================================================

print("--- Resource Usage Analysis Summary ---")

# Summary Table by Category
summary_by_category = df['resource_category'].value_counts().reset_index()
summary_by_category.columns = ['Resource Category', 'Count']
print("\n[Table 1] Overall Summary of Job Attempts by Resource Category:")
print(summary_by_category.to_markdown(index=False, numalign="left", stralign="left"))


# Analysis of Failures by Estimated File Size Regime
# Group failures into size buckets to find the weak spot.
failure_df = df[df['resource_category'].str.startswith('Failed')].copy()

# Define size bins for better visualization
size_bins = [0, 1, 5, 10, 20, np.inf]
size_labels = ['<1 GB (Min Cap)', '1-5 GB (Mid-Low)', '5-10 GB (Mid-High)', '10-20 GB (Max Cap)', '>20 GB (Over Max Cap)']

failure_df['size_regime'] = pd.cut(
    failure_df['estimated_size_gb'],
    bins=size_bins,
    labels=size_labels,
    right=False,
    include_lowest=True
)

# Cross-tabulate failure type against size regime
failure_crosstab = pd.crosstab(
    failure_df['size_regime'],
    failure_df['resource_category'],
    dropna=False
).fillna(0).astype(int)

print("\n[Table 2] Failures Broken Down by Estimated File Size and Failure Type:")
print(failure_crosstab.to_markdown(numalign="left", stralign="left"))


# Show examples of the key failure modes
print("\n--- Examples of Key Resource Issues ---")

# OOM Examples
print("\n[Example] Failed (OOM Kill):")
oom_example = df[df['resource_category'] == 'Failed (OOM Kill)']
if not oom_example.empty:
    print(oom_example.head(2)[['task_id', 'estimated_size_gb', 'requested_memory_gib', 'memory_gib', 'requested_runtime_seconds', 'runtime_seconds']].to_markdown(index=False, numalign="left", stralign="left"))

# Timeout Examples
print("\n[Example] Failed (Timeout):")
timeout_example = df[df['resource_category'] == 'Failed (Timeout)']
if not timeout_example.empty:
    print(timeout_example.head(2)[['task_id', 'estimated_size_gb', 'requested_memory_gib', 'memory_gib', 'requested_runtime_seconds', 'runtime_seconds']].to_markdown(index=False, numalign="left", stralign="left"))
    
# Wasted Memory Examples
print("\n[Example] Wastes Memory (Completed):")
wasted_mem_example = df[df['resource_category'] == 'Wastes Memory (Completed)']
if not wasted_mem_example.empty:
    print(wasted_mem_example.head(2)[['task_id', 'estimated_size_gb', 'requested_memory_gib', 'memory_gib', 'requested_runtime_seconds', 'runtime_seconds']].to_markdown(index=False, numalign="left", stralign="left"))

--- Resource Usage Analysis Summary ---

[Table 1] Overall Summary of Job Attempts by Resource Category:
| Resource Category       | Count   |
|:------------------------|:--------|
| Failed (Other/Unknown)  | 1162    |
| Wastes Both (Completed) | 965     |
| Wastes Time (Completed) | 865     |
| Efficient (Completed)   | 266     |
| Failed (Timeout)        | 179     |

[Table 2] Failures Broken Down by Estimated File Size and Failure Type:
| size_regime           | Failed (Other/Unknown)   | Failed (Timeout)   |
|:----------------------|:-------------------------|:-------------------|
| <1 GB (Min Cap)       | 0                        | 0                  |
| 1-5 GB (Mid-Low)      | 410                      | 62                 |
| 5-10 GB (Mid-High)    | 701                      | 17                 |
| 10-20 GB (Max Cap)    | 51                       | 100                |
| >20 GB (Over Max Cap) | 0                        | 0                  |

--- Examples of Key Resource Issues -